In [145]:
import librosa
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.metrics import categorical_accuracy
import time

In [2]:
commands = next(os.walk('./data'))[1]
commands

['down',
 'eight',
 'five',
 'four',
 'go',
 'left',
 'nine',
 'no',
 'off',
 'on',
 'one',
 'right',
 'seven',
 'six',
 'stop',
 'three',
 'two',
 'up',
 'yes',
 'zero']

In [3]:
data = pd.DataFrame(data=None, columns=['command','file'])
for command in commands:
    drty = './data/'+command
    files = next(os.walk(drty))[2]
    cmd = np.repeat(command,len(next(os.walk(drty))[2]))
    temp = pd.DataFrame({'command':cmd, 'file':files})
    data = data.append(temp)
    
data.tail()

,command,file
2371,zero,ffd2ba2f_nohash_1.wav
2372,zero,ffd2ba2f_nohash_2.wav
2373,zero,ffd2ba2f_nohash_3.wav
2374,zero,ffd2ba2f_nohash_4.wav
2375,zero,fffcabd1_nohash_0.wav


In [4]:
command_lookup = np.eye(len(commands))
y = np.array([command_lookup[commands.index(i)] for i in data.command])

In [25]:
X = data

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40245, 2)
(7103, 2)
(40245, 20)
(7103, 20)


In [141]:
def data_train_generator():
    for i in range(len(X_train)):
        filepath = './data/'+X_train.iloc[i].command+'/'+X_train.iloc[i].file
        x, sr = librosa.load(filepath)
        stft = np.abs(librosa.stft(x))
        mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=40).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        mel = np.mean(librosa.feature.melspectrogram(y=x, sr=sr).T,axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=sr).T, axis=0)
        ext_features = np.hstack([mfcc, chroma, mel, contrast, tonnetz])
        # X = np.empty((0,193))
        # X = np.vstack([features, ext_features])
        X = np.array(ext_features)
        Y = command_lookup[commands.index(X_train.iloc[i].command)]
        yield X.reshape(1,193), Y.reshape(1,20)

def data_test_generator():
    for i in range(len(X_test)):
        filepath = './data/'+X_test.iloc[i].command+'/'+X_test.iloc[i].file
        x, sr = librosa.load(filepath)
        stft = np.abs(librosa.stft(x))
        mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=40).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        mel = np.mean(librosa.feature.melspectrogram(y=x, sr=sr).T,axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=sr).T, axis=0)
        ext_features = np.hstack([mfcc, chroma, mel, contrast, tonnetz])
        # X = np.empty((0,193))
        # X = np.vstack([features, ext_features])
        X = np.array(ext_features)
        Y = command_lookup[commands.index(X_test.iloc[i].command)]
        yield X.reshape(1,193), Y.reshape(1,20)

In [142]:
train_gen = data_train_generator()
test_gen = data_test_generator()
print((next(train_gen)[0]).shape)
print((next(test_gen)[0]).shape)

(1, 193)
(1, 193)


In [59]:
# X_train.iloc[0].command
# print(len(X_train))
generator = data_train_generator()
i=10
for x, y in data_train_generator():
    print(x, y)
    if i>10:
        break
    i++

./data/off/8c7f81df_nohash_1.wav
./data/up/402e2977_nohash_1.wav
./data/zero/422d3197_nohash_0.wav
./data/off/35d1b6ee_nohash_1.wav
./data/five/4def68db_nohash_1.wav
./data/yes/ba676390_nohash_0.wav
./data/on/6cf5459b_nohash_0.wav
./data/eight/3f2b358d_nohash_2.wav
./data/seven/fe5c4a7a_nohash_0.wav
./data/two/25c14394_nohash_0.wav
./data/on/b6ebe225_nohash_0.wav
./data/one/65ec06e5_nohash_0.wav
./data/stop/541120c7_nohash_0.wav
./data/one/9799379a_nohash_0.wav
./data/right/937b433e_nohash_1.wav
./data/eight/8c7b691a_nohash_0.wav
./data/six/dbb40d24_nohash_4.wav
./data/nine/8eb4a1bf_nohash_1.wav
./data/two/8eb4a1bf_nohash_0.wav
./data/stop/8e05039f_nohash_1.wav
./data/zero/7e783e3f_nohash_0.wav
./data/six/d3badc9a_nohash_0.wav
./data/four/d0faf7e4_nohash_2.wav
./data/zero/6078eb0d_nohash_1.wav
./data/five/5aac2efa_nohash_0.wav
./data/up/f06190c1_nohash_0.wav
./data/four/742d6431_nohash_4.wav
./data/two/471a0925_nohash_2.wav
./data/six/8625475c_nohash_0.wav
./data/off/c634a189_nohash_3.

In [77]:
x_test_fit = []
y_test_fit = []
for i in range(len(X_test)):
    filepath = './data/'+X_test.iloc[i].command+'/'+X_test.iloc[i].file
    x, sr = librosa.load(filepath)
    stft = np.abs(librosa.stft(x))
    mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=x, sr=sr).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=sr).T, axis=0)
    ext_features = np.hstack([mfcc, chroma, mel, contrast, tonnetz])
    features = np.empty((0,193))
    features = np.vstack([features, ext_features])
    label = command_lookup[commands.index(X_test.iloc[i].command)]
    x_test_fit.append(features)
    y_test_fit.append(label)
    print(i)
    
x_test_fit = np.array(x_test_fit)
y_test_fit = np.array(y_test_fit)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


C:\Users\abjilani\AppData\Local\Continuum\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307


In [87]:
x_test_fit = x_test_fit.reshape((x_test_fit.shape[0], x_test_fit.shape[2]))
y_test_fit.shape

IndexError: tuple index out of range

In [146]:
nn = Sequential()
nn.add(Dense(units=300, input_dim=193, activation='relu'))
nn.add(Dense(units=len(commands), activation='softmax'))
nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=[categorical_accuracy])

In [ ]:
start_time = time.time()
nn.fit_generator(generator=train_gen, steps_per_epoch=len(X_train)/32, epochs=10, verbose=1, validation_data=(x_test_fit, y_test_fit))
end_time = time.time()
print(end_time, start_time)

Epoch 1/10
 114/1257 [=>............................] - ETA: 144s - loss: 15.6343 - categorical_accuracy: 0.0263     

C:\Users\abjilani\AppData\Local\Continuum\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


1258/1257 [==============================] - 174s - loss: 15.4080 - categorical_accuracy: 0.0437 - val_loss: 15.2717 - val_categorical_accuracy: 0.0525

In [113]:
len(X_train)/32

1257.65625